In [1]:
from pathlib import Path
import os,sys
from src.ImageClassifier.Constants import *
from src.ImageClassifier.Exception import CustomException
from src.ImageClassifier.loggers import logger
from src.ImageClassifier.Utils import Create_Directory,read_yaml_file,Save_Model
from dataclasses import dataclass
import tensorflow

## step:3) update entity files

In [2]:
#entity file is nothing but we r defining the parameters as class variable
#which i was used in yaml file along with it type of parameter
@dataclass
class PrepareBaseModelConfig():
    #defining the class variable init which was used in yaml file
    root_dir_path:Path
    base_model_path:Path
    update_base_model_path:Path
    include_top:bool
    weights:str
    input_shape:list
    classes:int
    batch_size:int
    epochs:int
    learning_rate:float

## step4) update the configuration manager file which is present in src/config/configuration.py


In [3]:
#In configuration manager file we read the yaml file and create directories accessing all the paths from it,
#and initializing those value to entity class parameter 

class ConfigurationManager():
    #creating constructor to initialize the yaml filepath
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        #now reading the yaml file
        self.config = read_yaml_file(config_filepath) #rtn values in configbox dictatonary
        self.param = read_yaml_file(param_filepath) #rtn values in configbox dictatonary

        #now creating artifacts directories
        Create_Directory([self.config.artifacts_root]) #artifact directory created in project structure

    #creating another method to initialize the value to the PrepareBaseModelConfig class variable to it and taking rtn as function
    def get_base_model_config(self) ->PrepareBaseModelConfig:
        self.config = self.config.prepare_base_model # rtn as configbox dictatonary

        #creating root directory for prepare_base_model key of yaml file
        Create_Directory([self.config.root_dir_path]) #artifacts/base_model directory created in project structure

        #now initializing the class variable value by creating an object PrepareBaseModelConfig class!!
        base_model_config = PrepareBaseModelConfig(
            root_dir_path=self.config.root_dir_path,
            base_model_path=self.config.base_model_path,
            update_base_model_path=self.config.update_base_model_path,
            include_top=self.param.include_top,
            weights=self.param.weights,
            input_shape=self.param.input_shape,
            classes=self.param.classes,
            batch_size=self.param.batch_size,
            epochs=self.param.epochs,
            learning_rate=self.param.learning_rate
        )

        return base_model_config

## step5) update the components files

In [4]:
#In this component files we gonna initialize the value to 
#preparebasemodel config class variable that having parameter as well as value init
#as well creating method to complete this task of preparebasemodel

class BaseModel():
    #creating construtor to initialize the variable to preparebasemodel class!!
    def __init__(self,config:PrepareBaseModelConfig):
        self.config = config


    #creating method for baseModel 
    def basemodel(self):
        #creating an object for base model
        self.basemodel = tensorflow.keras.applications.vgg16.VGG16(
            weights=self.config.weights,
            include_top=self.config.include_top,
            input_shape=self.config.input_shape,
            classes = self.config.classes
    
        )

        #now saving the base model to artifacts folder
        Save_Model(filepath=Path(self.config.base_model_path),model=self.basemodel) #this model rtn the vgg16 16 layers weights that include cnn layer+ relu layer+pooling+flatten+dense layer now using this model we r creating customize sequentual model

    @staticmethod
    def _prepare_full_custom_model(base_model,freeze,freeze_till,learning_rate,classes):
        #creating a sequential model class object
        sequential_model = tensorflow.keras.models.Sequential()

        #now adding all the vgg16 layers to this sequential model
        for layer in base_model.layers:
            sequential_model.add(layer)

        #now freezing all the cnn layer to sequential model except the dense layers
        if freeze:
            for layer in sequential_model.layers:
                layer.trainable = False #this line show freeze the weights and bias value during training

        elif (freeze_till is not None) and (freeze_till>0):
            for layer in sequential_model.layers[:freeze_till]:
                layer.trainable = False

        #now adding flattern layer to sequential model
        sequential_model.add(tensorflow.keras.layers.Flatten())

        #adding the dense layer to sequential model
        sequential_model.add(tensorflow.keras.layers.Dense(units = classes,activation="sigmoid"))

        logger.info(sequential_model)

        #now compiling the model
        sequential_model.compile(
            optimizer=tensorflow.keras.optimizers.Adam(learning_rate=learning_rate),
            loss = "binary_crossentropy",
            metrics=["accuracy"]
        )

        return sequential_model

        
    def update_base_model(self):
        #creating an object for sequential model
        self.sequential_model = self._prepare_full_custom_model(
            base_model = self.basemodel,
            freeze=True,
            freeze_till=None,
            learning_rate = self.config.learning_rate,
            classes = self.config.classes
        )

        #saving the sequential model to artifacts
        Save_Model(filepath=Path(self.config.update_base_model_path),model=self.sequential_model)

In [5]:
os.chdir('../')
%pwd

'd:\\DL MODEL\\ImageClassificationModel'

## step6) updating the training pipeline

In [6]:
try:
    #now creating an object of configuration manager class
    cm = ConfigurationManager()

    #now calling the method to get the configuration
    base_model_config = cm.get_base_model_config()

    #creating an object of BaseModel class
    bm = BaseModel(config=base_model_config)

    #now calling the method to get the model
    bm.basemodel()

    #now calling the method to get the updated sequential model
    bm.update_base_model()


except Exception as e:
    raise CustomException(e,sys)

[2024-09-10 15:46:57,789]-INFO-66-Reading the YAML file config\config.yaml
[2024-09-10 15:46:57,794]-INFO-69-YAML file read successfully: config\config.yaml
[2024-09-10 15:46:57,796]-INFO-66-Reading the YAML file params.yaml
[2024-09-10 15:46:57,802]-INFO-69-YAML file read successfully: params.yaml
[2024-09-10 15:46:57,804]-INFO-80-Creating Directory
[2024-09-10 15:46:57,806]-INFO-80-Creating Directory
[2024-09-10 15:46:59,339]-WARNING-359-Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


d:\DL MODEL\ImageClassificationModel\cnnmodel\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2024-09-10 15:46:59,781]-INFO-94-Model Save to Directory
[2024-09-10 15:47:00,029]-INFO-49-<keras.src.engine.sequential.Sequential object at 0x000002E3B5089970>
[2024-09-10 15:47:00,429]-INFO-94-Model Save to Directory
